In [69]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import math
import re

네이버 부동산 : https://land.naver.com/ 
참고 : https://cocoabba.tistory.com/58

https://m.land.naver.com/search/result/양천구신월동

In [53]:

# 검색할 키워드
keyword = "양천구신월동"

# 검색 URL
url = "https://m.land.naver.com/search/result/{}".format(keyword)

# 필요한 헤더 설정 (Referer 헤더는 실제 요청 시 사용한 referer URL로 설정)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Referer': 'https://m.land.naver.com/'
}

# GET 요청을 보냅니다
res = requests.get(url, headers=headers)
res.raise_for_status()

# 응답 데이터를 HTML로 파싱합니다
soup = (str)(BeautifulSoup(res.text, 'lxml'))

# HTML 내용 출력 (디버깅 용도)
print(soup)

<!DOCTYPE HTML>
<html data-user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="No-Cache" http-equiv="Cache-Control"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="index,nofollow" name="robots"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" id="_viewport" name="viewport"/>
<meta contents="always" name="referrer"/>
<title>네이버페이 부동산</title>
<link href="https://fin.land.naver.com/_front/image/apple-touch-icon.png" rel="apple-touch-icon" si

In [70]:

# 검색할 키워드
keyword = "양천구 신월동"

# 검색 URL
url = "https://m.land.naver.com/search/result/{}".format(keyword)

# 필요한 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Referer': 'https://m.land.naver.com/'
}

# GET 요청을 보냅니다
res = requests.get(url, headers=headers)
res.raise_for_status()

# 응답 데이터를 HTML로 파싱합니다
soup = str(BeautifulSoup(res.text, 'lxml'))

# 필요한 값 추출
value = soup.split("filter: {")[1].split("}")[0].replace(" ","").replace("'","")

print(value)

lat = (float)(value.split("lat:")[1].split(",")[0])
lon = (float)(value.split("lon:")[1].split(",")[0])
z = (int)(value.split("z:")[1].split(",")[0])
cortarNo = value.split("cortarNo:")[1].split(",")[0]
rletTpCds = value.split("rletTpCds:")[1].split(",")[0]
tradTpCds = value.split("tradTpCds:")[1].split()[0]




lat:37.527925,
lon:126.8342,
z:14,
cortarNo:1147010300,
cortarNm:신월동,
rletTpCds:*,
tradTpCds:A1:B1:B2



In [71]:
def calculate_bounding_box(lat, lon, zoom):
    # Earth's radius in meters
    R = 6378137
    # Approximate bounding box in meters for zoom level 14 (usually covers around 1km)
    bounding_box_half_side_length = 1000  # 1km / 2

    # Convert latitude and longitude from degrees to radians
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    
    # Offset calculations
    lat_offset = (bounding_box_half_side_length / R) * (180 / math.pi)
    lon_offset = (bounding_box_half_side_length / (R * math.cos(lat_rad))) * (180 / math.pi)
    
    # Calculate bounding box coordinates
    btm = lat - lat_offset
    lft = lon - lon_offset
    top = lat + lat_offset
    rgt = lon + lon_offset
    
    return btm, lft, top, rgt

In [72]:
btm, lft, top, rgt = calculate_bounding_box(lat, lon, z)
print(f"btm: {btm}, lft: {lft}, top: {top}, rgt: {rgt}")

btm: 37.51894184715881, lft: 126.82287274611544, top: 37.5369081528412, rgt: 126.84552725388455


In [73]:
# clusterList URL
clusterList_URL = "https://m.land.naver.com/cluster/clusterList?view=atcl&cortarNo={}&rletTpCd={}&tradTpCd={}&z={}&lat={}&lon={}&btm={}&lft={}&top={}&rgt={}"\
    .format(cortarNo, rletTpCds, tradTpCds, z, lat, lon, btm, lft, top, rgt)
print(clusterList_URL)

# clusterList URL로 GET 요청
res2 = requests.get(clusterList_URL, headers=headers)
res2.raise_for_status()

# print(res2.text)

# JSON 데이터를 파싱
try:
    data = res2.json()
    # print(json.dumps(data, indent=2, ensure_ascii=False))
except json.JSONDecodeError as e:
    print("JSON 데이터를 파싱할 수 없습니다:", e)
    print("응답 내용:", res2.text)
    
values = data['data']['ARTICLE']
# print(values)


https://m.land.naver.com/cluster/clusterList?view=atcl&cortarNo=1147010300&rletTpCd=*&tradTpCd=A1:B1:B2&z=14&lat=37.527925&lon=126.8342&btm=37.51894184715881&lft=126.82287274611544&top=37.5369081528412&rgt=126.84552725388455


In [74]:

# 결과를 저장할 리스트
result_list = []

# 큰 원으로 구성되어 있는 전체 매물그룹(values)을 load 하여 한 그룹씩 세부 쿼리 진행
for v in values:
    lgeo = v['lgeo']
    count = v['count']
    z2 = v['z']
    lat2 = v['lat']
    lon2 = v['lon']

    len_pages = math.ceil(count / 20) + 1
    for idx in range(1, len_pages):
        
        articleList_URL = "https://m.land.naver.com/cluster/ajax/articleList?""itemId={}&mapKey=&lgeo={}&showR0=&" \
               "rletTpCd={}&tradTpCd={}&z={}&lat={}&""lon={}&totCnt={}&cortarNo={}&page={}"\
            .format(lgeo, lgeo, rletTpCds, tradTpCds, z2, lat2, lon2, count,cortarNo, idx)
        # print(articleList_URL)
        res3 = requests.get(articleList_URL, headers=headers)
        res3.raise_for_status()
                
        try:
            data2 = res3.json()
            articles = data2['body']
            for article in articles:
                result_list.append(article)
        except json.JSONDecodeError as e:
            print("JSON 데이터를 파싱할 수 없습니다:", e)
            print("응답 내용:", res3.text)

# pandas DataFrame으로 변환
df = pd.DataFrame(result_list)

# CSV 파일로 저장
df.to_csv(f'{keyword}_list.csv', index=False, encoding='utf-8-sig')

print("CSV 파일로 저장되었습니다.")
            
        


JSON 데이터를 파싱할 수 없습니다: Expecting value: line 1 column 1 (char 0)
응답 내용: <!DOCTYPE HTML>
<html lang="ko">
<head>
    <link rel='shortcut icon' href='https://m.naver.com/favicon.ico' type='image/x-icon'>
    <meta name="format-detection" content="telephone=no">
    <meta http-equiv="Content-Type" content="text/html;charset=utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no">
    <meta name="format-detection" content="telephone=no">
    <title>네이버페이 부동산</title>
    <meta http-equiv="Content-Script-Type" content="text/javascript">
    <meta http-equiv="Content-Style-Type" content="text/css">
    <meta name="referrer" contents="always">
    <link rel="stylesheet" href="https://ssl.pstatic.net/static.land/static/space/css/deploy/20240528140238/min/space.min.css" type="text/css"><script src="https://ssl.pstatic.net/static.land/static/space/js/deploy/20240528140238/min/jquery_common.min.js" type="text/javascri

In [75]:
df

,atclNo,cortarNo,atclNm,atclStatCd,rletTpCd,uprRletTpCd,rletTpNm,tradTpCd,tradTpNm,vrfcTpCd,...,minMviFee,maxMviFee,etRoomCnt,tradePriceHan,tradeRentPrice,tradeCheckedByOwner,cpLinkVO,dtlAddrYn,dtlAddr,sellrNm
0,2426149609,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B2,월세,MOBL,...,0,0,0,,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,,NaN
1,2425857390,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B1,전세,MOBL,...,0,0,0,,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,,NaN
2,2425798538,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,A1,매매,MOBL,...,0,0,0,,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,NaN
3,2426163175,1147010300,신정뉴타운롯데캐슬,R0,A01,A01,아파트,A1,매매,OWNER,...,0,0,0,,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,NaN
4,2426165236,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B2,월세,OWNER,...,0,0,0,,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,2423046023,1147010300,일반상가,R0,D02,D02,상가,A1,매매,NONE,...,0,0,0,,0,False,"{'cpId': 'KAR', 'mobileArticleLinkTypeCode': '...",N,,NaN
796,2423038496,1147010300,해태,R0,A01,A01,아파트,A1,매매,NONE,...,0,0,0,,0,False,"{'cpId': 'KAR', 'mobileArticleLinkTypeCode': '...",N,,NaN
797,2422599957,1147010300,서울,R0,A01,A01,아파트,A1,매매,NONE,...,0,0,0,,0,False,"{'cpId': 'KAR', 'mobileArticleLinkTypeCode': '...",N,,NaN
798,2422626066,1147010300,장수마을,R0,A01,A01,아파트,A1,매매,NONE,...,0,0,0,,0,False,"{'cpId': 'KAR', 'mobileArticleLinkTypeCode': '...",N,,NaN


In [13]:
df.head()

,atclNo,cortarNo,atclNm,atclStatCd,rletTpCd,uprRletTpCd,rletTpNm,tradTpCd,tradTpNm,vrfcTpCd,...,etRoomCnt,tradePriceHan,tradeRentPrice,tradeCheckedByOwner,cpLinkVO,dtlAddrYn,dtlAddr,sameAddrMaxPrc2,sameAddrMinPrc2,sellrNm
0,2425857390,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B1,전세,MOBL,...,0,,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,,NaN,NaN,NaN
1,2425798538,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,A1,매매,MOBL,...,0,,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,NaN,NaN,NaN
2,2426086256,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B1,전세,OWNER,...,0,,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,NaN,NaN,NaN
3,2426074781,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,A1,매매,OWNER,...,0,,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,,NaN,NaN,NaN
4,2426078377,1147010300,스카이캐슬,R0,A01,A01,아파트,B1,전세,OWNER,...,0,,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,NaN,NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1810 entries, 0 to 1809
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   atclNo               1810 non-null   object 
 1   cortarNo             1810 non-null   object 
 2   atclNm               1810 non-null   object 
 3   atclStatCd           1810 non-null   object 
 4   rletTpCd             1810 non-null   object 
 5   uprRletTpCd          1810 non-null   object 
 6   rletTpNm             1810 non-null   object 
 7   tradTpCd             1810 non-null   object 
 8   tradTpNm             1810 non-null   object 
 9   vrfcTpCd             1810 non-null   object 
 10  flrInfo              1810 non-null   object 
 11  prc                  1810 non-null   int64  
 12  rentPrc              1810 non-null   int64  
 13  hanPrc               1810 non-null   object 
 14  spc1                 1810 non-null   object 
 15  spc2                 1810 non-null   o

In [23]:
df1 = df['atclStatCd'].value_counts()
df1

atclStatCd
R0    1810
Name: count, dtype: int64

In [68]:
df = pd.read_csv(f'real_estate_listings.csv')
df

,atclNo,cortarNo,atclNm,atclStatCd,rletTpCd,uprRletTpCd,rletTpNm,tradTpCd,tradTpNm,vrfcTpCd,...,etRoomCnt,tradePriceHan,tradeRentPrice,tradeCheckedByOwner,cpLinkVO,dtlAddrYn,dtlAddr,sameAddrMaxPrc2,sameAddrMinPrc2,sellrNm
0,2425857390,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B1,전세,MOBL,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
1,2425798538,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,A1,매매,MOBL,...,0,NaN,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,NaN,NaN,NaN,NaN
2,2426086256,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B1,전세,OWNER,...,0,NaN,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,NaN,NaN,NaN,NaN
3,2426074781,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,A1,매매,OWNER,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
4,2426078377,1147010300,스카이캐슬,R0,A01,A01,아파트,B1,전세,OWNER,...,0,NaN,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1805,2420913843,1147010300,일반상가,R0,D02,D02,상가,B2,월세,DOC,...,0,NaN,0,False,"{'cpId': 'NEONET', 'mobileArticleLinkTypeCode'...",N,NaN,NaN,NaN,NaN
1806,2420919734,1147010300,단독,R0,C03,C03,단독/다가구,A1,매매,DOC,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
1807,2420914814,1147010300,일반상가,R0,D02,D02,상가,B2,월세,DOC,...,0,NaN,0,False,"{'cpId': 'NEONET', 'mobileArticleLinkTypeCode'...",N,NaN,NaN,NaN,NaN
1808,2420912201,1147010300,빌딩,R0,D03,D03,건물,A1,매매,DOC,...,0,NaN,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,NaN,NaN,NaN,NaN


In [51]:
df1 = df['cpCnt'].unique()
df1

array([3, 8, 4, 5, 6, 2, 7, 1, 9], dtype=int64)

In [43]:
df1 = df['cpLinkVO'].unique()
df1

array(["{'cpId': 'sunbang', 'mobileArticleLinkTypeCode': 'NONE', 'mobileBmsInspectPassYn': 'Y', 'pcArticleLinkUseAtArticleTitle': False, 'pcArticleLinkUseAtCpName': False, 'mobileArticleLinkUseAtArticleTitle': False, 'mobileArticleLinkUseAtCpName': False}",
       "{'cpId': 'bizmk', 'mobileArticleLinkTypeCode': 'NONE', 'mobileBmsInspectPassYn': 'Y', 'pcArticleLinkUseAtArticleTitle': False, 'pcArticleLinkUseAtCpName': False, 'mobileArticleLinkUseAtArticleTitle': False, 'mobileArticleLinkUseAtCpName': False}",
       "{'cpId': 'NEONET', 'mobileArticleLinkTypeCode': 'NONE', 'mobileBmsInspectPassYn': 'Y', 'pcArticleLinkUseAtArticleTitle': False, 'pcArticleLinkUseAtCpName': False, 'mobileArticleLinkUseAtArticleTitle': False, 'mobileArticleLinkUseAtCpName': False}",
       "{'cpId': 'asil', 'mobileArticleUrl': 'https://asil.kr/app/sale_info.jsp?UID=', 'mobileArticleLinkTypeCode': 'ALL', 'mobileBmsInspectPassYn': 'Y', 'pcArticleLinkUseAtArticleTitle': False, 'pcArticleLinkUseAtCpName': False,

In [45]:
mob1_rows = df[df['cpLinkVO'] == "{'cpId': 'sunbang', 'mobileArticleLinkTypeCode': 'NONE', 'mobileBmsInspectPassYn': 'Y', 'pcArticleLinkUseAtArticleTitle': False, 'pcArticleLinkUseAtCpName': False, 'mobileArticleLinkUseAtArticleTitle': False, 'mobileArticleLinkUseAtCpName': False}"]
mob1_rows

,atclNo,cortarNo,atclNm,atclStatCd,rletTpCd,uprRletTpCd,rletTpNm,tradTpCd,tradTpNm,vrfcTpCd,...,etRoomCnt,tradePriceHan,tradeRentPrice,tradeCheckedByOwner,cpLinkVO,dtlAddrYn,dtlAddr,sameAddrMaxPrc2,sameAddrMinPrc2,sellrNm
0,2425857390,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B1,전세,MOBL,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
3,2426074781,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,A1,매매,OWNER,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
34,2426061021,1147010300,신정뉴타운두산위브,R0,A01,A01,아파트,A1,매매,OWNER,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
51,2425935498,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B1,전세,OWNER,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
68,2425925920,1147010300,동방(주상복합),R0,A01,A01,아파트,A1,매매,NDOC1,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671,2423306093,1147010300,빌라,R0,C02,C03,빌라,A1,매매,DOC,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
1675,2423316181,1147010300,대,R0,E03,E03,토지,A1,매매,DOC,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
1721,2422690745,1147010300,복합상가,R0,D02,D02,상가,A1,매매,DOC,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN
1760,2421970373,1147010300,빌라,R0,C02,C03,빌라,A1,매매,DOC,...,0,NaN,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,NaN,NaN,NaN,NaN


In [19]:
unique_atclNm_series = pd.Series(df1)
unique_atclNm_series

0           목동센트럴아이파크위브
1                 스카이캐슬
2             신정뉴타운롯데캐슬
3                  신안약수
4                신월신도1차
             ...       
101    훼미리(166-27,주상복합)
102                신월해밀
103                골든캐슬
104            신월동리버스타운
105                라온미브
Length: 106, dtype: object

In [14]:
print(df['atclNm'].describe())

count     1810
unique     106
top       일반상가
freq       369
Name: atclNm, dtype: object


In [76]:

df.isnull().sum()

atclNo                    0
cortarNo                  0
atclNm                    0
atclStatCd                0
rletTpCd                  0
uprRletTpCd               0
rletTpNm                  0
tradTpCd                  0
tradTpNm                  0
vrfcTpCd                  0
flrInfo                   0
prc                       0
rentPrc                   0
hanPrc                    0
spc1                      0
spc2                      0
direction                19
atclCfmYmd                0
repImgUrl              1416
repImgTpCd             1416
repImgThumb            1416
lat                       0
lng                       0
atclFetrDesc            142
tagList                   0
bildNm                    0
minute                    0
sameAddrCnt               0
sameAddrDirectCnt         0
sameAddrHash           1351
sameAddrMaxPrc         1351
sameAddrMinPrc         1351
cpid                      0
cpNm                      0
cpCnt                     0
rltrNm              

## 데이터 전처리 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(f'양천구신월동_list.csv')

In [4]:
## 6개 칼럼과 100개의 랜덤 데이터 출력
selected_columns = ["atclNo", "atclNm", "rletTpNm", "prc", "lat", "lng"]
df = df[selected_columns].sample(100).reset_index(drop=True)

In [5]:
df

,atclNo,atclNm,rletTpNm,prc,lat,lng
0,2426130767,목동센트럴아이파크위브,아파트,60000,37.517669,126.844850
1,2425438083,상가주택,상가주택,250000,37.517523,126.842308
2,2423609940,스카이캐슬,아파트,97000,37.520542,126.844645
3,2425722784,단독,단독/다가구,15000,37.518080,126.842841
4,2422558122,일반상가,상가,108000,37.517721,126.842002
...,...,...,...,...,...,...
95,2423233490,빌라,빌라,25500,37.521391,126.844122
96,2426060226,신성미소지움3차,아파트,63000,37.522890,126.841696
97,2425566480,금용,아파트,83000,37.520768,126.842091
98,2426059502,신정뉴타운롯데캐슬,아파트,81000,37.518923,126.847877


In [6]:
df.to_csv(f'양천구_신월동_100.csv', index=False, encoding='utf-8-sig')

In [88]:
df.info() # 데이터프레임의 정보 출력

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   atclNo    100 non-null    int64  
 1   atclNm    100 non-null    object 
 2   rletTpNm  100 non-null    object 
 3   prc       100 non-null    int64  
 4   lat       100 non-null    float64
 5   lng       100 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 4.8+ KB


In [90]:
df.isnull().sum() # 데이터프레임의 결측치 정보 출력

atclNo      0
atclNm      0
rletTpNm    0
prc         0
lat         0
lng         0
dtype: int64

In [7]:
df = pd.read_csv('양천구_신월동_100.csv',encoding='utf-8')
df

,atclNo,atclNm,rletTpNm,prc,lat,lng
0,2426130767,목동센트럴아이파크위브,아파트,60000,37.517669,126.844850
1,2425438083,상가주택,상가주택,250000,37.517523,126.842308
2,2423609940,스카이캐슬,아파트,97000,37.520542,126.844645
3,2425722784,단독,단독/다가구,15000,37.518080,126.842841
4,2422558122,일반상가,상가,108000,37.517721,126.842002
...,...,...,...,...,...,...
95,2423233490,빌라,빌라,25500,37.521391,126.844122
96,2426060226,신성미소지움3차,아파트,63000,37.522890,126.841696
97,2425566480,금용,아파트,83000,37.520768,126.842091
98,2426059502,신정뉴타운롯데캐슬,아파트,81000,37.518923,126.847877
